# 燃焼実験のログを取る

燃焼実験で取ったログの生データから単位付きの値に変換して簡単なグラフにするまでの処理です   
このファイルを実行するには、Pythonの環境が必要です。colabやJupyter Notebookで実行しましょう。  
by michael  

In [ ]:
inputfilepath="C:\\n0628\\1658-bento\\LOG-0000001.csv"# ここを変えて使います /でも //でも\でもエラーになるみたいなのでフォルダの区切りは\\でやります
outputfilepath=inputfilepath.replace(".csv","-newconverted.csv")

# analysis.py

YutoGCNさん作のプログラムです。一番下の行のmain()の引数をいじって使います。  
第一引数が読み込むファイルのパス、第二引数が出力するファイルのパスです。  
このコードで変換した後のデータを推進も電装もいじくることになります。  
新旧どちらのロードセルを使うかによって、bin2ThrustTestを2か4に変えてください。

In [ ]:
#!/usr/bin/env python

# -*- coding: utf-8 -*-

import csv
import sys
import math

voltageref5v = 4.99

def main(source,target):
    with open(source) as fr:
        reader = csv.reader(fr)
        with open(target, 'w', encoding='utf_8_sig',newline="") as fw:
             writer = csv.writer(fw)
             writer.writerow(['書き込み待ちデータ数','データ取得開始時','データ取得終了時','推力[N]','圧力1[Pa]','圧力2[Pa]','圧力3[Pa]','圧力4[Pa]','低域温度1','低域温度2','低域温度3','高域温度1','高域温度2'])
             for row in reader:
                # 1行ずつ読み込み
                writer.writerow([row[0],row[1],row[2],str(bin2ThrustTest4(int(row[3]))),str(bin2dataPressure(int(row[5]))),str(bin2dataPressure(int(row[6]))),str(bin2dataPressure(int(row[7]))),str(bin2dataPressure(int(row[8]))),str(bin2dataTempLow(int(row[11]))),str(bin2dataTempLow(int(row[12]))),str(bin2dataTempLow2(int(row[13]))),str(bin2dataTempHigh(int(row[14]))),str(bin2dataTempHigh(int(row[15]))),row[19]])
                #この行で使う関数を決めています。とくにbin2ThrustTestは1,2,3,4があり、1,2は旧ロードセル、3,4は新ロードセルに対応しています。
    

def bin2ADCvoltage(binary):
    return voltageref5v * float(binary) / 4096

'''
理論値
250lbf = 1112.06 N について，5.1* 20 * 1e-3 V の出力が得られる
'''
def bin2Thrust(binary):
    sensorOutputVoltage = bin2ADCvoltage(binary)/47.0
    return 1112.06 * sensorOutputVoltage / (voltageref5v * 20.0 * 1e-3) # N

'''
実験値
ADCの値 x に対して，力 y[N] は，y = 0.5952 * x - 20.177
'''
def bin2ThrustTest1(binary):
    return 0.5952 * binary - 20.177

def bin2ThrustTest2(binary): #古いロードセルでやった時の校正値、こっちがbin2ThrustTest1より正確だと思われる
    return 0.5807 * binary - 24.098
def bin2ThrustTest3(binary): #ナメた方法でやった時の校正値
    return 2.4699*binary-126.66
def bin2ThrustTest4(binary): #コンクリブロック法でやった時の校正値、こっちがbin2ThrustTest3より正確だと思われる
    return 2.3896*binary-115.47

def bin2dataPressure(binary):
    sensorOutputVoltage = 2.0 * bin2ADCvoltage(binary)
    return sensorOutputVoltage * 1e6 # Nm^-2

def bin2dataTempHigh(binary):
    R = 10000
    return bin2dataTemp(binary,R)

def bin2dataTempLow(binary):
    R = 100
    return bin2dataTemp(binary,R)

def bin2dataTempLow2(binary):
    R=100
    return bin2dataTempNew(binary,R)

def bin2dataTemp(binary,R):
    # サーミスタの抵抗値を計算
    v = bin2ADCvoltage(binary)
    R_T = R * v / (voltageref5v -v)

    if R_T == 0:
        return 0

    # サーミスタの抵抗値から温度を計算
    B = 3435 # B係数
    R0 = 10000
    T0 = 25 + 273.15

    T = 1/(1/B * math.log(R_T/R0)+ 1/T0)
    T -= 273.15
    return T
def bin2dataTempNew(binary,R):
    # サーミスタの抵抗値を計算
    v = bin2ADCvoltage(binary)
    R_T = R * v / (voltageref5v -v)

    if R_T == 0:
        return 0

    # サーミスタの抵抗値から温度を計算
    B = 4126 # B係数
    R0 = 10000
    T0 = 25 + 273.15

    T = 1/(1/B * math.log(R_T/R0)+ 1/T0)
    T -= 273.15
    return T

'''
def bin2dataTemp(binary,R):
    if(binary == 0):
        return -273
    Voltage = bin2ADCvoltage(binary)
    return 1/(1/(25+273)+math.log(((Voltage/(5.1-Voltage))*R)/10000)/4126)-273
'''

#if __name__ == "__main__":
#    args = sys.argv
main(inputfilepath,outputfilepath)

### インポート（このコードを動かす前にpandasとmatplotlibを入れてください）

##### pandas,matplotlibのインストール方法
colabなら以下のコードをセルに入れて実行してください。  
```python
!pip install pandas matplotlib
```
他の環境の人はpandasやmatplotlibの使い方をググってください。

以下のコードを実行してライブラリをインポートします

In [ ]:
import pandas as pd
import matplotlib as plt
plt.rcParams['font.family'] = "MS Gothic" # 日本語フォントを指定

### 変数dataに読み込み

ここの表の見出しを覚えておいて後で使います。indexの行は実質的にはindexではなくなっています

In [ ]:
data=pd.read_csv(outputfilepath,header=0)
data.reset_index(inplace=True)
data=data.shift(axis=1)
data.drop(columns="index")
data["index"]=0
print(data)

以下　このデータをいじくってグラフにしたりします

### データの編集（例）

推力が最大値の0.05倍以下の点のみをプロット、ファイルに出力(このファイルを推進に渡すとよいでしょう)

In [ ]:
tcutdata=data[(data["推力[N]"]>data["推力[N]"].max()*0.11)]
tcutdata.plot(y='推力[N]',x='データ取得開始時',kind='scatter',xlabel='time',ylabel='thrust')
tcutdata.to_csv("20250331output2SJIS.csv",encoding="SHIFT-JIS") #ファイル出力です、このファイルと同じディレクトリに吐き出されます

温度のグラフをプロット

In [ ]:

data.plot(y='低域温度1',x='データ取得開始時',kind='scatter',xlabel='time',ylabel='低域温度1')

データを時間で切り抜き、プロット、ファイルに出力

In [ ]:
cutdata=data.loc[(data["データ取得開始時"]>0)]# 時間で切るときはこんな感じです
cutdata.plot(y='圧力4[Pa]',x='データ取得開始時',kind='scatter',xlabel='time',ylabel='pressure')
cutdata.to_csv("output2SJIS.csv",encoding="SHIFT-JIS") 